![Podmanlogo](Pictures/podman-logo.png)

# Managing container images

In the previous section of this workshop we've seen how to run container images pulled from different registries but, what if I want to create a container image for my own application? No worries, podman has your back.

Podman can be used for building your own container images and push them to a container registry. In order to do it you need to first build a file called "Containerfile" and then used the "podman build" command to create your image out of the content of the Containerfile.

Lets see an example.

## Creating a new container image

Lets begin by creating a new Container file with the definition of our container image:

In [ ]:
HostIP={{ hostvars[inventory_hostname]['IP-WKSHP-Podman101'] }}

In [ ]:
%login $HostIP

In [ ]:
cat > Containerfile << EOF
FROM docker.io/redhat/ubi9
RUN dnf -y update \
 && dnf -y install httpd \
 && dnf clean all
COPY index.html /var/www/html/index.html
EXPOSE 80
ENTRYPOINT /usr/sbin/httpd -DFOREGROUND
EOF
cat Containerfile

The output of the command is the content of our newly created Containerfile. Let's review it line by line.

First we have specified the "FROM" option, this line is mandatory, you need it to build your container image. With this line we specify the base container image that we will use for our application, in our example we've chosen a redhat/ubi9 image as base for our application. Choosing an image that you can trust is critical to maintain a secure environment, if your base image has vulnerabilities it may affect your application and potentially your whole enviroment. This is why we have chosen a Red Hat Universal Base Image (UBI) image for Red Hat Enterprise Linux 9 (RHEL9).

Red Hat creates UBI images that are analyzed often and gives you information about the vulnerabilities that it may have and how to solve them. There are many options for your UBI images, from the standard RHEL 8 or RHEL 9 to a much more secure micro image and even images for specific types of applications (like http, nodejs, etc). You can find a detailed list of all the CVE's and security status of Red Hat's UBI images in [Red Hat's catalog](https://catalog.redhat.com/software/containers/search).
Also, if you're a Red Hat customer you will have full support for these images. But, if you're not, do not worry as some of this images are also uploaded to docker.io so you can access them and have enterprise grade security in your containers for free.

![redhatcatalog](Pictures/certifiedcontainers.png)

With that we know that our application will be running on top of a trusted and secured container image. Now we need to add our software layer. We used the "RUN" option to specify the command that needs to be executed to install the required software in our container image. If you don't have any experience with RHEL, we're just using the dnf command to update all packages, install httpd and clean the cache (this last step is used only to have lighter container image).

Once the httpd package is installed we would like to customize the message that our containerized web server will show to the world. For that we used the "COPY" command. This command will copy a file from our local directory into the container that we are building. For our container we are passing a file index.html from our local directory to the directory /var/www/html/ inside the container. We do this because we know that httpd uses that file to define the content to be posted through our web server.
If we want it to work we will need to create the index.html file so it can the be copied inside the container once we build it. Let's do it.

In [ ]:
echo "Hello from HPE Dev and Red Hat teams :)" > index.html
cat index.html

You see a message as output from the previous command, that is the message that our web server will be showing.

Next line in our Containerfile is using the "EXPOSE" instruction. The EXPOSE instruction informs Podman that the container listens on the specified network ports at runtime, in our case the port 80.
Hint: Actually the port opend is defined by the configuration of the daemon being run inside the container. In our case apache is configured to listen on port 80, which it starts to listen on. Nevertheless it is good practice to declare the ports exposed correctly. Other frameworks might rely on this directive.

Last, an ENTRYPOINT allows you to configure the command that your container will run when started, in our case we want it to run the httpd binary to make sure our web server is running.

Next thing we need to do is building our newly created container image and that is quite a simple task. We just need to run the following command:

In [ ]:
podman build --tag myhttpd:v1 -f ./Containerfile

As you can see we passed two options to the podman build command. The first one, the tag option, specifies the name and tag of the container, in our case the name is myhttpd and v1 is the tag. Next, we used the "-f" option to specify the Containerfile that we want to use for building our container, by default podman build would take any file called Containerfile in the current directory but it's a best practice to use the "-f" option to make sure we use the correct file.

Our container image is now stored locally in our server and we can see it by running the following command:

In [ ]:
podman images

Our brand new container image is listed in the repository localhost/myhttpd and the tag v1 as we specified. Now our image is limited to be used in our local system, if we wanted to use it from other platforms we should upload the image to a container registry by running the command "podman push". We wont be doing this in this workshop, but you can look at all the details on how to use it by running "man podman-push".

As a last step let's run our image and check if it works fine.

In [ ]:
podman run -d -p {{ HTTPPORT }}:{{ HTTPPORT }} --name myhttpd myhttpd:v1

We called our container "myhttpd" and pay attention to the option "-p". With this option we specified that the port 8080 of our system will be mapped to the port 80 of our container, if you remember correctly we specified in our Containerfile that our application was using the port 80 and now if anything within our network reaches the port 8080 of our RHEL machine it will be redirected to the port 80 of the container.

Lets test it by running a curl command against our local machine and the output should be the text we specified previously in our index.html file:

In [ ]:
curl -s $HostIP:{{ HTTPPORT }}

# Containers are ephemeral

You may have heard it, "containers are ephemeral", but what does it mean exactly? Let's use our newly created container for explaining it.

Our container is showing the message "Hello from HPE Dev and Red Hat teams :)" when we run a curl to its exposed port. Now the container is running and we are going to change that message by accesing interactively to the container.

First open a new tab by clicking on the "+" sign on top of the notebook.

![interactive-container-1](Pictures/interactive-container-1.png)

In the launcher tab select the terminal option.

![interactive-container-2](Pictures/interactive-container-2.png)

Then ssh into the remote podman machine 16.31.86.187. Remember to use your student number which may be different from the one in the picture. Also, if you get asked, accept the connection to the host by typing "yes".

![interactive-container-3](Pictures/interactive-container-3.png)

Now run the command "podman exec -it myhttpd /bin/bash", it will open an interactive shell to the container. Check that the content of the file by running the command "cat /var/www/html/index.html".

![ephemeral-1](Pictures/ephemeral-1.png)

You have confirmed you logged into the right container. Lets modify the content of the file by running the command "echo 'Am I ephemeral?' > /var/www/html/index.html".

![ephemeral-2](Pictures/ephemeral-2.png)

Check it worked out running again "cat /var/www/html/index.html".

![ephemeral-3](Pictures/ephemeral-3.png)

Demonstrate the web server is showing the new message:

In [ ]:
curl -s $HostIP:{{ HTTPPORT }}

As you can see the output shows the new message. You can now close the tab that we opened to interact with the container.

The big question is: what happens if the container is stopped? And what if it's removed? Lets see.

First stop the container:

In [ ]:
podman stop myhttpd

Check the container was stopped by looking at its status:

In [ ]:
podman ps -a

Now restart the container and use curl again to learn what message is showing.

In [ ]:
podman start myhttpd
curl -s $HostIP:{{ HTTPPORT }}

It's showing the same message! So we learnt that you can stop and start a container and the information within it will persist. But what happens if we remove and recreate it?
Lets test it. We will start stopping and deleting the container.

In [ ]:
podman stop myhttpd
podman rm myhttpd

Now we will recreate the container and check the message.

In [ ]:
podman run -d --rm -p {{ HTTPPORT }}:{{ HTTPPORT }} --name myhttpd myhttpd:v1
curl -s $HostIP:{{ HTTPPORT }}

Well, as you can see the message it's showing is the one we had at the beginning. This happens because we have actually created a new container from the same image, so it will show the content provided within the container image.

There are scenarios in which you want to persist data. For example, if you have v1 and v2 of your containerized database you will have two container images (one for each version). For updating your application you will need to remove your old v1 containers and create new v2 ones, therefore all data will be lost and, of course, you don't want to lose database data. If you want to persist data when doing this kind of operations you will need to use volumes and connect them to the containers instead of just using the storage layered within the container.

Usually you want to work with containers in a way that they can be destroyed and recreated without changing the behaviour of the application. This brings agility and predictability as it's easier to know the expected behaviour of the application. Remember that one of the reasons to containerize applications is to package them with their dependencies so they behave the same independantly of the system they are deployed to. This is why most times you will see containers being removed and recreated instead of stopped and restarted.
As a consecuence you may experience that it's not common to interactively modify your application by accessing to the container. This is specially true for production environments, while developers may want to sneek in the containers in their local tests.

Using volumes with Podman will be covered in a future Podman 201 workshop, keep yourself up to date with HPE Dev workshops to access to it as soon as it's released!

# Cleanup

In [ ]:
podman stop --all
podman rm --all
podman rmi localhost/myhttpd:v1
rm ./Containerfile ./index.html

In [ ]:
%logout

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#631f61;"></i>&nbsp;&nbsp;Next Steps

# Lab 3 : Podman security

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="4-WKSHP-Podman-security.ipynb" target="New" title="Next LAB: Podman security"><i class="fas fa-chevron-circle-right" style="color:#631f61;"></i></a></h2>

</br>
 <a href="2-WKSHP-Podman-hello-world.ipynb" target="New" title="Back: Podman hello-world"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="4-WKSHP-Podman-security.ipynb" target="New" title="Next: Podman security"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#631f61;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
